In [ ]:
# as the column "year" have just one value, we are going to drop it.
df.drop(["Year"], axis=1, inplace=True)


In [ ]:
# changing types:
df[['DepTime', 'ArrTime']] = df[['DepTime', 'ArrTime']].astype(int)

# to_change changing type from "objet" to "category"
to_change = ["Month", "DayofMonth", "DayOfWeek", "FlightNum", "Origin",
             "Dest", "TailNum", "UniqueCarrier", "CancellationCode", "Cancelled", "Diverted"]

for x in to_change:
    df[x] = df[x].astype("category")

to_date = ["CRSArrTime", "CRSDepTime", "DepTime", "ArrTime"]

# to_date changing type to date

for x in to_date:
    df[x] = df[x].astype(str).str.zfill(4)
    df[x] = df[x].astype(str).str[:2] + ':' + df[x].astype(str).str[2:] + ':00'
    df.loc[df[x] == "24:00:00", x] = "00:00:00"
    df[x] = pd.to_datetime(df[x], format="%H:%M:%S").dt.time


In [ ]:
# where on time or delay new column
# where delayed = 1
# and on time = 0

df['with_delay'] = np.where(df['ArrDelay'] > 0, 1, 0)
df['with_delay'] = df['with_delay'].astype("category")


In [ ]:
# let's drop the columns of Delays that are not ArrDelay, because ArrDelay is the sum of all others
df = df.drop(['CarrierDelay', 'WeatherDelay', 'NASDelay',
             'SecurityDelay', 'LateAircraftDelay'], axis=1)


In [ ]:
# keep values cancelled ==1 (flights cancelled) in a diferent dataframe.
df_cancelled = df.loc[df['Cancelled'] == 1]
# keep values diverted ==1 (flights diverted) in a diferent dataframe.
df_diverted = df.loc[df['Diverted'] == 1]

# New dataframe with out cancelled diverted and ontime values
df_analisys = df.loc[(df['Cancelled'] == 0) & (
    df['Diverted'] == 0) & (df['with_delay'] == 1)].copy()
# clean memory
del df


In [ ]:
# As we aren't going to use it, we can drop "Cacelled","CancellationCode","Diverted","with_delayt" columns from our new dataframe
df_analisys.drop(columns=["Cancelled", "CancellationCode",
                 "Diverted", "with_delay"], inplace=True)


In [ ]:
# filter numerical features
num_features = df_analisys.select_dtypes(exclude=["category"])


In [ ]:
# check for null values"
num_features.isnull().sum()


In [ ]:
# Cyclical features encoding: converting the original time to their corresponding cosine and sine values
to_transform = ["DepTime", "CRSDepTime", "ArrTime", "CRSArrTime"]

for column in to_transform:
    num_features[[f'h_{column}', f'm_{column}', f's_{column}']] = num_features[column].astype(
        str).str.split(':', expand=True).astype(int)
    num_features.drop([f's_{column}'], axis=1, inplace=True)
    num_features[f'x_{column}'] = num_features[f'h_{column}'] * \
        60 + num_features[f'm_{column}']
    num_features.drop([f'h_{column}'], axis=1, inplace=True)
    num_features.drop([f'm_{column}'], axis=1, inplace=True)
    num_features[f'x_{column}'] = 2 * math.pi * \
        num_features[f'x_{column}'] / num_features[f'x_{column}'].max()
    num_features[f"cos_{column}"] = np.cos(num_features[f'x_{column}'])
    num_features[f"sin_{column}"] = np.sin(num_features[f'x_{column}'])
    num_features.drop([f'x_{column}'], axis=1, inplace=True)
    num_features.drop([column], axis=1, inplace=True)


In [ ]:
# keep label of the airlines
num_features["true_label_names"] = df_analisys["UniqueCarrier"]


In [ ]:
# create a label encoder obj-
label_encoder = LabelEncoder()
true_labels = label_encoder.fit_transform(num_features["true_label_names"])
# keep true labels
num_features["true_labels"] = true_labels


In [ ]:
sns.set_context("talk")
plt.style.use('ggplot')

fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(20, 10))

pd.crosstab(data["kmeans_3_RobustScaler"], data["true_label_names"]).T.plot.bar(stacked=True,ax=axes[0])
axes[0].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0,fontsize = 'small')
axes[0].set_title("Unique carriers 3 cluster\n(With RobusScaler)")

pd.crosstab(data["kmeans_3_StandardScaler"], data["true_label_names"]).T.plot.bar(stacked=True,ax=axes[1])
axes[1].legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0,fontsize = 'small')
axes[1].set_title("Unique carriers 3 cluster\n(With StandarScaler)")

plt.tight_layout()
plt.show()